<a href="https://colab.research.google.com/github/ghostpasha01/Data_Analysis/blob/main/Customer_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go 
import plotly.express as px 
import seaborn as sns
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Lasso
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression


In [ ]:
df=pd.read_csv("C://Users//Hp//Desktop//Veri setler(Yeni Site)//CustomerChurn//telecom_customer_churn.csv")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.isnull().sum()


In [ ]:
df1=df.copy()

In [ ]:
df1.head()

In [ ]:
df1.drop(['Customer ID','Total Refunds','Zip Code','Latitude', 'Longitude','Churn Category', 'Churn Reason'],axis='columns',inplace=True)

In [ ]:
df1.shape

In [ ]:
df1.dtypes

In [ ]:
features=df1.columns
for feature in features:
    print(f'{feature}--->{df[feature].nunique()}')

In [ ]:
features=df1.columns
for feature in features:
    if features.dtype==object:
     print(f'{feature}--->{df[feature].nunique()}')

In [ ]:
# get percent of null values 
df1.isnull().sum() / df1.shape[0]

In [ ]:
#cleaing function
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame)
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [ ]:
df1=df1.interpolate()

In [ ]:
df1=df1.dropna()

In [ ]:
df['Unlimited Data']

In [ ]:
number_columns=['Age','Number of Dependents','Number of Referrals','Tenure in Months',
                'Avg Monthly Long Distance Charges','Avg Monthly GB Download','Monthly Charge',
                'Total Charges','Total Extra Data Charges',
                'Total Long Distance Charges','Total Revenue']

In [ ]:
def unique_values_names(df):
    for column in df:
        if df[column].dtype=='object':
            print(f'{column}:{df[column].unique()}')

In [ ]:
unique_values_names(df1)


In [ ]:
fig = px.histogram(df1, x = 'Age')
fig.show()

In [ ]:
df1.hist(figsize=(12,12), xrot=30)


In [ ]:
df1['Age']

In [ ]:
Customer_Stayed=df1[df1['Customer Status']=='Stayed'].Age
Customer_Churned=df1[df1['Customer Status']=='Churned'].Age
Customer_Joined=df1[df1['Customer Status']=='Joined'].Age


plt.xlabel('Age')
plt.ylabel('customers numbers')
plt.hist([Customer_Stayed,Customer_Churned,Customer_Joined], color=['black','red','blue'],label=['Stayed','Churned','Joined'])


plt.title('customers behavior ',fontweight ="bold")
plt.legend()

In [ ]:
data  = df1.corr()
plt.figure(figsize = (20,10))
sns.heatmap(data, annot = True)

In [ ]:
fig, ax = plt.subplots(4,3, figsize = (15,15))
for i, subplot in zip(number_columns, ax.flatten()):
    sns.boxplot(x = 'Customer Status', y =i , data = df1, ax = subplot)
    

In [ ]:
fig = px.density_heatmap(df1, x='Age', y='Total Charges')
fig.show()

In [ ]:
df1.columns

In [ ]:
pd.crosstab(df['Customer Status'], df['Married']).plot(kind='bar')



In [ ]:
pd.crosstab(df['Customer Status'], df['Gender']).plot(kind='bar')


In [ ]:
df1['Payment Method'].unique()


In [ ]:
#Create dictionary with role / data key value pairs
Roles = {}
for j in df1['Payment Method'].unique():
    Roles[j] = df1[df1['Payment Method'] == j]

In [ ]:
Roles.keys()

In [ ]:
Roles['Credit Card']


In [ ]:
len(Roles)

In [ ]:
off = df1['Offer'].value_counts()
off

In [ ]:
fig = go.Figure([go.Bar(x=off.index, y=off.values)])
fig.show()

In [ ]:
# Now let us consider credit card  -Case as an example
df1_off = Roles['Credit Card'].Offer.value_counts()
df1_off

In [ ]:
fig = go.Figure([go.Bar(x= df1_off.index, y=df1_off.values)])
fig.show()

In [ ]:
# Comparing offers  by type of payment
#########################################
fig = go.Figure(layout=go.Layout(title= go.layout.Title(text="Comparing offer  by type of payment")))
#changed from role selection to selection 1
fig.add_trace(go.Bar(name= 'Selection 1', x= off.index, y=(off.values/ off.values.sum())))

buttons = []

#added button for all data comparison
buttons.append(dict(method='restyle',
                        label= 'All Samples',
                        visible=True,
                        args=[{'y':[df1.Offer.value_counts().values/df1.Offer.value_counts().values.sum()],
                               'x':[df1.Offer.value_counts().index],
                               'type':'bar'}, [0]], # the [0] at the end lets us know they are for the first trace
                        )
                  )

for i in list(Roles.keys())[0:]:
    buttons.append(dict(method='restyle',
                        label= i,
                        visible=True,
                        args=[{'y':[Roles[i].Offer.value_counts().values/Roles[i].Offer.value_counts().values.sum()],
                               'x':[Roles[i].Offer.value_counts().index],
                               'type':'bar'}, [0]], # the [0] at the end lets us know they are for the first trace
                        )
                  )

fig.add_trace(go.Bar(name= 'Selection 2',x= off.index, y=(off.values/ off.values.sum())))

buttons2 = []
#added button for all data comparison
buttons2.append(dict(method='restyle',
                        label= 'All Samples',
                        visible=True,
                        args=[{'y':[df1.Offer.value_counts().values/df1.Offer.value_counts().values.sum()],
                               'x':[df1.Offer.value_counts().index],
                               'type':'bar'}, [1]], # the [0] at the end lets us know they are for the first trace
                        )
                  )

for i in list(Roles.keys())[0:]:
    buttons2.append(dict(method='restyle',
                        label= i,
                        visible=True,
                        args=[{'y':[Roles[i].Offer.value_counts().values/Roles[i].Offer.value_counts().values.sum()],
                               'x':[Roles[i].Offer.value_counts().index],
                               'type':'bar'}, [1]], # the [1] at the end lets us know they are for the first trace
                        )                        #literally figured that out by just experimenting 
                  )
# adjusted dropdown placement 
#found out updatemenus take a dictionary of buttons and allow you to format how the dropdowns look etc.
# https://plotly.com/python/dropdowns/
button_layer_1_height = 1.23
updatemenus = list([
    dict(buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.11,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"),
    dict(buttons=buttons2,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.71,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top")])
    
fig.update_layout( updatemenus=updatemenus)
#added annotations next to dropdowns 
fig.update_layout(
    annotations=[
        dict(text="Selection 1", x=0, xref="paper", y=1.15, yref="paper",
                             align="left", showarrow=False),
        dict(text="Selection 2", x=0.65, xref="paper", y=1.15,
                             yref="paper", showarrow=False)
    ])
fig.update_xaxes(categoryorder= 'array',categoryarray= ['None' 'Offer E' 'Offer D' 'Offer A' 'Offer B' 'Offer C'])


In [ ]:
df1 = df1.rename(columns = {'Customer Status':'Customer_Status'})


In [ ]:
Roles1 = {}
for k in df1['Customer_Status'].unique():
    Roles1[k] = df1[df1['Customer_Status'] == k]
Roles1.keys()

In [ ]:
df1_state = Roles1['Stayed'].Offer.value_counts()
df1_state

In [ ]:
# Comparing offers  by customer status
#########################################
fig = go.Figure(layout=go.Layout(title= go.layout.Title(text="Comparing offer  by customer status")))
#changed from role selection to selection 1
fig.add_trace(go.Bar(name= 'Selection 1', x= off.index, y=(off.values/ off.values.sum())))

buttons = []

#added button for all data comparison
buttons.append(dict(method='restyle',
                        label= 'All Samples',
                        visible=True,
                        args=[{'y':[df1.Offer.value_counts().values/df1.Offer.value_counts().values.sum()],
                               'x':[df1.Offer.value_counts().index],
                               'type':'bar'}, [0]], # the [0] at the end lets us know they are for the first trace
                        )
                  )

for i in list(Roles1.keys())[0:]:
    buttons.append(dict(method='restyle',
                        label= i,
                        visible=True,
                        args=[{'y':[Roles1[i].Offer.value_counts().values/Roles1[i].Offer.value_counts().values.sum()],
                               'x':[Roles1[i].Offer.value_counts().index],
                               'type':'bar'}, [0]], # the [0] at the end lets us know they are for the first trace
                        )
                  )

fig.add_trace(go.Bar(name= 'Selection 2',x= off.index, y=(off.values/ off.values.sum())))

buttons2 = []
#added button for all data comparison
buttons2.append(dict(method='restyle',
                        label= 'All Samples',
                        visible=True,
                        args=[{'y':[df1.Offer.value_counts().values/df1.Offer.value_counts().values.sum()],
                               'x':[df1.Offer.value_counts().index],
                               'type':'bar'}, [1]], # the [0] at the end lets us know they are for the first trace
                        )
                  )

for i in list(Roles1.keys())[0:]:
    buttons2.append(dict(method='restyle',
                        label= i,
                        visible=True,
                        args=[{'y':[Roles1[i].Offer.value_counts().values/Roles1[i].Offer.value_counts().values.sum()],
                               'x':[Roles1[i].Offer.value_counts().index],
                               'type':'bar'}, [1]], # the [1] at the end lets us know they are for the first trace
                        )                        #literally figured that out by just experimenting 
                  )
# adjusted dropdown placement 
#found out updatemenus take a dictionary of buttons and allow you to format how the dropdowns look etc.
# https://plotly.com/python/dropdowns/
button_layer_1_height = 1.23
updatemenus = list([
    dict(buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.11,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"),
    dict(buttons=buttons2,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.71,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top")])
    
fig.update_layout( updatemenus=updatemenus)
#added annotations next to dropdowns 
fig.update_layout(
    annotations=[
        dict(text="Selection 1", x=0, xref="paper", y=1.15, yref="paper",
                             align="left", showarrow=False),
        dict(text="Selection 2", x=0.65, xref="paper", y=1.15,
                             yref="paper", showarrow=False)
    ])
fig.update_xaxes(categoryorder= 'array',categoryarray= ['None' 'Offer E' 'Offer D' 'Offer A' 'Offer B' 'Offer C'])

In [ ]:
df1.replace({"Gender":{'Female':0,'Male':1}},inplace=True)


In [ ]:
yes_and_no=[  'Paperless Billing', 'Unlimited Data', 
       'Streaming Movies', 'Streaming Music',  'Streaming TV',
       'Premium Tech Support', 'Device Protection Plan', 'Online Backup', 'Online Security',
       'Multiple Lines',  'Married']
for i in yes_and_no:
    df1.replace({'No':0,'Yes':1},inplace=True)

In [ ]:
df1.replace({"Phone Service":{'Yes':1}},inplace=True)


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df1.Customer_Status = le.fit_transform(df1.Customer_Status)

In [ ]:
df1 = pd.get_dummies(data=df1, columns=['Payment Method','Contract','Internet Type','Offer','City'])


In [ ]:
cols_to_scale = ['Age','Number of Dependents','Number of Referrals','Tenure in Months','Avg Monthly Long Distance Charges','Avg Monthly GB Download','Monthly Charge', 'Total Charges',
       'Total Extra Data Charges', 'Total Long Distance Charges','Total Revenue']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df1[cols_to_scale] = scaler.fit_transform(df1[cols_to_scale])

In [ ]:
X = df1.drop('Customer_Status',axis='columns')
y = df1['Customer_Status']


In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=5)

In [ ]:
len(X_train)


In [ ]:
X_train[:10]


In [ ]:
import xgboost as xgb


In [ ]:
from xgboost import XGBClassifier
from sklearn import svm
model_params = {
     
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
    'naive_bayes_gaussian': {
        'model': GaussianNB(),
        'params': {}
    },
    'naive_bayes_multinomial': {
        'model': MultinomialNB(),
        'params': {}
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'criterion': ['gini','entropy'],
        }    
    },
       'XGB_Classifier':{
        'model':XGBClassifier(),
        'params':{
            'base_score':[0.5]
            
        }
    }, 
        
     
      }

In [ ]:
from sklearn.model_selection import GridSearchCV
scores = []
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=cv, return_train_score=False)
    clf.fit(X,y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

In [ ]:
reg=XGBClassifier()
reg.fit(X_train, y_train)

In [ ]:
reg.score(X_test, y_test)


In [ ]:
y_predicted = reg.predict(X_test)
y_predicted[:5]

In [ ]:
y_test[:5]



In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predicted)
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predicted)
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predicted)
